In [2]:
import pandas as pd
import numpy as np
from copy import deepcopy
import sys
import numpy as np
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

from importlib import reload
sys.path.append("D:/DST/")
import cathay_db as db
import utils as ut
import financial_statement as fs

reload(ut)
reload(fs)
import matplotlib.pyplot as plt
import datetime
import os

# set max display rows
pd.set_option("display.max_rows", 200)

# set max display columns
pd.set_option("display.max_columns", 100)

# Set the float format to display without scientific notation
pd.options.display.float_format = '{:.4f}'.format

# 1. Read data from feather file

In [3]:
df_factor_all = pd.read_feather('data/df_factor_all.feather')

In [4]:
# # following is data from tej
# df_adjusted_price = pd.read_feather('data/tej_adjusted_price.feather')

# # following is data from cmoney
# df_price = pd.read_feather('data/cmoney_price.feather')
# df_company_info = pd.read_feather('data/cmoney_company_info.feather')

# ticker_list = np.sort(pd.unique(df_company_info['股票代號']))

# df_price = ut.cmoney_data_clean_up(df_price)
# df_price = deepcopy(df_price[df_price['股票代號'].isin(ticker_list)])

# df_adjusted_price = ut.tej_data_clean_up(df_adjusted_price)
# df_adjusted_price = deepcopy(df_adjusted_price[df_adjusted_price['股票代號'].isin(ticker_list)])

In [5]:
df_factor_all['y'] = df_factor_all.groupby('ticker', as_index=False, group_keys=False)['price'].pct_change(-62)

In [6]:
def get_rebalance_date(date_list, start_date, end_date, freq='Q'):
    next_four_seasons = [start_date]
    rebalance_date = start_date
    i = 0
    while rebalance_date < end_date:
        if freq == 'Q':
            rebalance_date = start_date + pd.DateOffset(months=3 * (i + 1))
        elif freq == 'M':
            rebalance_date = start_date + pd.DateOffset(months=1 * (i + 1))
        elif freq == 'Y':
            rebalance_date = start_date + pd.DateOffset(years=1 * (i + 1))

        # check if rebalance_date is in date_list
        if rebalance_date in date_list:
            next_four_seasons.append(rebalance_date)
        else:
            # if not, find the next date in date_list
            for date in date_list:
                if date > rebalance_date:
                    # chang np.datetime64 to pd.Timestamp
                    rebalance_date = pd.Timestamp(date)
                    next_four_seasons.append(rebalance_date)
                    break
        i += 1
    return next_four_seasons

In [7]:
date_list = np.sort(pd.unique(df_factor_all['date']))
rebalance_date_lst = get_rebalance_date(date_list, pd.to_datetime('2003-01-15'), pd.to_datetime('2020-12-31'), freq='Q')

In [8]:
factor_columns = df_factor_all.columns.tolist()
factor_columns.remove('date')
factor_columns.remove('ticker')
factor_columns.remove('price')
factor_columns.remove('y')
print(factor_columns)

['excess_return', 'asset_qoq', 'asset_yoy', 'ppe_qoq', 'ppe_yoy', 'ni_qoq', 'ni_yoy', 'roe', 'roe_yoy', 'roe_4q_sum', 'roe_4q_sum_yoy', 'tobins_q', 'ocd / asset', 'dividend_1Y_sum_yield', 'dividend_2Y_sum_yield', 'dividend_3Y_sum_yield', 'last_dividend_yield']


In [21]:
target_cols = ['y']
# df_factor_all = df_factor_all.dropna(subset=factor_columns + target_cols, how='any')

In [36]:
df_factor_all_train = df_factor_all[df_factor_all['date'] == rebalance_date_lst[4]]

In [38]:
target_cols = ['y']
df_factor_all_train.dropna(subset=factor_columns)

,date,ticker,price,excess_return,asset_qoq,asset_yoy,ppe_qoq,ppe_yoy,ni_qoq,ni_yoy,roe,roe_yoy,roe_4q_sum,roe_4q_sum_yoy,tobins_q,ocd / asset,dividend_1Y_sum_yield,dividend_2Y_sum_yield,dividend_3Y_sum_yield,last_dividend_yield,y
1021,2004-01-15,1101,4.6527,0.1298,0.9924,0.9864,0.9862,0.9691,1.3250,3.1984,0.0609,0.0172,0.0450,6.2793,0.7606,0.0543,0.0055,0.0055,0.0055,0.0000,0.0225
6781,2004-01-15,1102,4.5430,0.1946,0.9965,1.0318,0.9903,0.9281,1.7324,4.6060,0.1017,0.8827,0.0795,1.3969,0.9242,0.0314,0.0248,0.0347,0.0470,0.0000,-0.0776
12541,2004-01-15,1103,10.8789,-0.0057,0.9955,0.9763,0.9974,0.9905,1.0085,3.6623,0.0642,7.7575,0.0558,0.4243,1.0368,-0.0007,0.0000,0.0000,0.0000,0.0000,0.0939
18301,2004-01-15,1104,4.2977,0.0760,1.0147,1.0213,0.9923,0.9738,0.6141,-4.5399,0.1098,-4.1850,0.0630,5.8487,0.5575,0.0184,0.0000,0.0000,0.0071,0.0000,-0.0310
27167,2004-01-15,1108,4.9724,0.4056,0.9769,1.0186,0.9762,0.8988,-2.9580,-6.6779,0.0374,0.1319,0.0084,-0.5047,0.7667,0.0277,0.0122,0.0122,0.0122,0.0000,-0.2969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7820537,2004-01-15,9937,8.9335,-0.0006,1.0162,1.0607,1.0315,1.1808,2.3275,0.9728,0.1706,0.3122,0.1419,-0.0219,1.0445,0.1017,0.0630,0.0925,0.1063,0.0000,0.0496
7837231,2004-01-15,9940,4.5306,0.0059,1.0445,1.2684,1.0025,1.2577,1.6248,2.2196,0.4909,0.8766,0.3194,0.1987,3.4756,0.2071,0.0288,0.0452,0.0539,0.0000,-0.1576
7842991,2004-01-15,9941,6.1850,0.0342,1.0088,1.0474,0.9747,1.0250,0.9736,1.0740,0.1701,0.2173,0.1662,0.1245,0.8103,0.0945,0.0673,0.1347,0.1953,0.0000,-0.0660
7853731,2004-01-15,9943,12.9067,0.0705,0.8080,1.1284,1.0108,1.1318,-5.0646,1.1613,0.1859,8.9687,0.0841,-0.4771,1.5349,0.2252,0.0469,0.1250,0.1445,0.0000,0.0449


In [40]:
df_factor_all_train[df_factor_all_train['ticker'] == '2330']

,date,ticker,price,excess_return,asset_qoq,asset_yoy,ppe_qoq,ppe_yoy,ni_qoq,ni_yoy,roe,roe_yoy,roe_4q_sum,roe_4q_sum_yoy,tobins_q,ocd / asset,dividend_1Y_sum_yield,dividend_2Y_sum_yield,dividend_3Y_sum_yield,last_dividend_yield,y
1653396,2004-01-15,2330,25.4291,0.1254,1.0533,1.0126,0.9821,0.8841,1.2931,4.7999,0.2023,4.6029,0.1127,0.4752,3.6870,0.2020,0.0000,0.0000,0.0000,0.0000,0.0984


In [35]:
for col_name in factor_columns:
    print(col_name, len(df_factor_all_train[df_factor_all_train[col_name].isna() == False]))

excess_return 1715
asset_qoq 1671
asset_yoy 1673
ppe_qoq 1671
ppe_yoy 1673
ni_qoq 1671
ni_yoy 1671
roe 1671
roe_yoy 1671
roe_4q_sum 1671
roe_4q_sum_yoy 1664
tobins_q 1504
ocd / asset 1673
dividend_1Y_sum_yield 1679
dividend_2Y_sum_yield 1617
dividend_3Y_sum_yield 1574
last_dividend_yield 1696


In [25]:
start_index = 4
# rebalance_date_lst
model_save_path = 'D:/Projects/0_HIDIV/models'

for i in range(start_index, len(rebalance_date_lst)):
    # expanding window
    train_date = rebalance_date_lst[0:i]
    rebalance_date = rebalance_date_lst[i]

    print("train_date : {}".format(train_date))
    print("rebalance_date : {}".format(rebalance_date))

    # 訓練股票
    param_search = dict(
        learning_rate=[0.01, 0.05, 0.1],
        max_depth=[3, 5, 7],
        n_estimators=[5, 10, 50, 100, 200],
    )

    df_factor_all_train = df_factor_all[df_factor_all['date'].isin(train_date)]
    
    model_stock = XGBRegressor()
    gscv = GridSearchCV(
        model_stock, param_search, refit=True, scoring="neg_root_mean_squared_error"
    )

    gscv.fit(df_factor_all_train[factor_columns], df_factor_all_train[target_cols])
    # timestamp to string format for saving model
    rebalance_date = rebalance_date.strftime("%Y%m%d")
    gscv.best_estimator_.save_model(f"{model_save_path}/{rebalance_date}.json")


train_date : [Timestamp('2003-01-15 00:00:00'), Timestamp('2003-04-15 00:00:00'), Timestamp('2003-07-15 00:00:00'), Timestamp('2003-10-15 00:00:00')]
rebalance_date : 2004-01-15 00:00:00
train_date : [Timestamp('2003-01-15 00:00:00'), Timestamp('2003-04-15 00:00:00'), Timestamp('2003-07-15 00:00:00'), Timestamp('2003-10-15 00:00:00'), Timestamp('2004-01-15 00:00:00')]
rebalance_date : 2004-04-15 00:00:00
train_date : [Timestamp('2003-01-15 00:00:00'), Timestamp('2003-04-15 00:00:00'), Timestamp('2003-07-15 00:00:00'), Timestamp('2003-10-15 00:00:00'), Timestamp('2004-01-15 00:00:00'), Timestamp('2004-04-15 00:00:00')]
rebalance_date : 2004-07-15 00:00:00
train_date : [Timestamp('2003-01-15 00:00:00'), Timestamp('2003-04-15 00:00:00'), Timestamp('2003-07-15 00:00:00'), Timestamp('2003-10-15 00:00:00'), Timestamp('2004-01-15 00:00:00'), Timestamp('2004-04-15 00:00:00'), Timestamp('2004-07-15 00:00:00')]
rebalance_date : 2004-10-15 00:00:00
train_date : [Timestamp('2003-01-15 00:00:00'), 

d:\Projects\0_HIDIV\venv\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
180 fits failed out of a total of 225.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
13 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Projects\0_HIDIV\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Projects\0_HIDIV\venv\Lib\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "d:\Projects\0_HIDIV\venv\Lib\site-packages\xgboost\sklearn.py", line 988, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
                       

XGBoostError: [10:53:39] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\data\data.cc:1104: Check failed: valid: Input data contains `inf` or `nan`

['excess_return',
 'asset_qoq',
 'asset_yoy',
 'ppe_qoq',
 'ppe_yoy',
 'ni_qoq',
 'ni_yoy',
 'roe',
 'roe_yoy',
 'roe_4q_sum',
 'roe_4q_sum_yoy',
 'tobins_q',
 'ocd / asset',
 'dividend_1Y_sum_yield',
 'dividend_2Y_sum_yield',
 'dividend_3Y_sum_yield',
 'last_dividend_yield']